In [18]:
import os

WEIGHTS_DIR = 'weights'
ORIGINAL_TRAIN_DIR = os.path.join('data', 'train')
REDUCED_TRAIN_DIR = os.path.join('data', 'train_reduced')
REDUCED_VAL_DIR = os.path.join('data', 'val_reduced')

In [3]:
# Data preparation

import glob
import random
import shutil

os.makedirs(WEIGHTS_DIR, exist_ok=True)

N_train = 1250
N_val = 125

if not os.path.isdir(REDUCED_TRAIN_DIR):
    for cls in ('cat', 'dog'):
        dst_train_dir = os.path.join(REDUCED_TRAIN_DIR, cls)
        dst_val_dir = os.path.join(REDUCED_VAL_DIR, cls)
        os.makedirs(dst_train_dir, exist_ok=True)
        os.makedirs(dst_val_dir, exist_ok=True)
        files = glob.glob(os.path.join(ORIGINAL_TRAIN_DIR, cls + '*'))
        random.shuffle(files)
        for f in files[:N_train]:
            shutil.copy(f, dst_train_dir)
        for f in files[N_train:N_train + N_val]:
            shutil.copy(f, dst_val_dir)

In [19]:
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD
print(keras.__version__)

2.1.1


In [20]:
base_model = VGG16(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# Add a binary classification layer (sigmoid)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

height, width = 224, 224
datagen = ImageDataGenerator(
    rotation_range=45.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.1,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

batch_size = 32

train_generator = datagen.flow_from_directory(
        REDUCED_TRAIN_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        REDUCED_VAL_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

Found 2500 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


In [12]:
from keras.callbacks import TensorBoard

tb_callback = TensorBoard(log_dir='tb_log')

training = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        callbacks=[tb_callback])

Epoch 1/50
79/78 [==============================] - 68s 858ms/step - loss: 0.5647 - acc: 0.7677 - val_loss: 0.3001 - val_acc: 0.8800
Epoch 2/50
79/78 [==============================] - 28s 358ms/step - loss: 0.2291 - acc: 0.9089 - val_loss: 0.2157 - val_acc: 0.9200
Epoch 3/50
79/78 [==============================] - 27s 345ms/step - loss: 0.1864 - acc: 0.9186 - val_loss: 0.1706 - val_acc: 0.9360
Epoch 4/50
79/78 [==============================] - 27s 346ms/step - loss: 0.1658 - acc: 0.9332 - val_loss: 0.1644 - val_acc: 0.9360
Epoch 5/50
79/78 [==============================] - 27s 343ms/step - loss: 0.1431 - acc: 0.9426 - val_loss: 0.1494 - val_acc: 0.9520
Epoch 6/50
79/78 [==============================] - 27s 339ms/step - loss: 0.1337 - acc: 0.9414 - val_loss: 0.1243 - val_acc: 0.9560
Epoch 7/50
79/78 [==============================] - 29s 363ms/step - loss: 0.1213 - acc: 0.9494 - val_loss: 0.1255 - val_acc: 0.9560
Epoch 8/50
79/78 [==============================] - 29s 365ms/step - 

In [13]:
model.save('trained_model.h5')

----

In [27]:
from keras.models import load_model
model = load_model('trained_model.h5')

In [34]:
# Inferência em uma imagem
from keras.preprocessing import image
from skimage.io import imshow
import numpy as np

img = image.load_img('data/train/dog.100.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)